In [1]:
# import packages
import pickle
import praw
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
reddit = praw.Reddit(client_id='R30fcxAZLJ7Wyw',
                     client_secret='8Zf87krHYa_zXRXPWEF8hK2Twfs',
                     user_agent='project1'
                    )
print(reddit.read_only)

True


In [3]:
flairs={0:'Politics',1:'Non-Political',2:'AskIndia', 3:"[R]eddiquette", 4:'Science/Technology', 5:'Policy/Economy', 6:'Business/Finance', 7:'Scheduled', 8:'Sports', 9:'Food', 10:'Photography', 11:'AMA', 12:'Coronavirus'}

In [4]:
def preprocess_input(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = ''
    for w in word_tokens:
        if w not in stop_words:
            if w.isalnum():
                w = w.lower()
                filtered_sentence = filtered_sentence + ' ' + w
    filtered_sentence = " ".join(filtered_sentence.split())
    return(filtered_sentence)

In [5]:
def detect_flair(url,loaded_model):

    submission = reddit.submission(url=url)

    data = {}

    data['title'] = submission.title
    data['url'] = submission.url

    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
    data["comment"] = comment
    data['title'] = preprocess_input(data['title'])
    data['comment'] = preprocess_input(data['comment'])
    data['combine'] = data['title'] + ' ' + data['comment']

    return flairs[loaded_model.predict([data['title']])[0]]

In [8]:
# load the model from disk
loaded_model = pickle.load(open("../model/tmodel.pkl", 'rb'))

In [9]:
url = "https://www.reddit.com/r/india/comments/g62sa1/i_find_this_extremely_strange_album_on_imgur_some/"
detect_flair(url, loaded_model )

'Business/Finance'